# 11/05 - Pair Programming Limpieza 5 - Lidia y Tatiana

In [8]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [9]:
df = pd.read_csv("./limpio con nan.csv", index_col=0)
df

,year,type,country,sex_,age,species_,fatal_(y/n),date
0,2018,Boating,usa,F,57.0,white_shark,N,25-Jun-2018
1,2018,Unprovoked,usa,F,11.0,unspecified,N,18-Jun-2018
2,2018,Invalid,usa,M,48.0,unspecified,N,09-Jun-2018
3,2018,Unprovoked,australia,M,NaN,unspecified,N,08-Jun-2018
4,2018,Provoked,mexico,M,NaN,tiger_shark,N,04-Jun-2018
...,...,...,...,...,...,...,...,...
6278,0,Unprovoked,australia,M,NaN,unspecified,Y,Before 1906
6279,0,Unprovoked,australia,M,NaN,unspecified,Y,Before 1903
6281,0,Unprovoked,usa,M,NaN,unspecified,Y,1900-1905
6282,0,Unprovoked,panama,M,NaN,unspecified,Y,1883-1889


In [10]:
df.isnull().sum()

year              0
type              4
country          49
sex_            562
age            2807
species_          0
fatal_(y/n)     532
date              0
dtype: int64

In [11]:
df["age"].unique()

array([57., 11., 48., nan, 18., 52., 15., 12., 32., 10., 21., 34., 30.,
       60., 33., 29., 54., 41., 37., 56., 19., 25., 69., 38., 55., 35.,
       46., 45., 14., 40., 28., 20., 24., 26., 49., 22.,  7., 31., 17.,
       13., 42.,  3.,  8., 50., 16., 82., 73., 68., 51., 39., 58., 47.,
       61., 65., 36., 66., 43.,  9., 72., 59.,  6., 27., 64., 23., 71.,
       44., 62., 63., 70., 53., 77., 74.,  5., 86., 84., 75., 87., 67.,
        1.,  2., 81., 78.])

In [12]:
#simple imputer
from sklearn.impute import SimpleImputer

In [13]:
#interative imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [14]:
#knn imputer
from sklearn.impute import KNNImputer

In [15]:
nulos = pd.DataFrame((df.isnull().sum() * 100) / df.shape[0]).reset_index()
nulos.columns = ["columna", "porcentaje"]
nulos

,columna,porcentaje
0,year,0.000000
1,type,0.064133
2,country,0.785634
3,sex_,9.010742
4,age,45.005612
5,species_,0.000000
6,fatal_(y/n),8.529742
7,date,0.000000


# EJERCICIO 1

- **Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.**

In [16]:
#Primero comprueba cuantos nulos existen en "age"
df["age"].isnull().sum()

2807

In [17]:
#Primero imputamos con estrategia media
imputer = SimpleImputer(strategy='mean', missing_values=np.nan)
#Donde voy a trabajar
imputer = imputer.fit(df[['age']])
#Aplico la estrategia transformandolo y creando una nueva columna
df['new_age'] = imputer.transform(df[['age']])

In [18]:
#Compruebo si el valor de nulos es ahora 0
df["new_age"].isnull().sum()

0

- **Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.**

>💡 ** Pista** 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [19]:
#Primero comprueba cuantos nulos existen en "sex_"
df["sex_"].isnull().sum()

562

In [20]:
#Primero imputamos con estrategia moda
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
#Donde voy a trabajar
imputer = imputer.fit(df[['sex_']])
#Aplico la estrategia transformandolo y creando una nueva columna
df['new_sex'] = imputer.transform(df[['sex_']])

In [21]:
#Compruebo si el valor de nulos es ahora 0
df["new_sex"].isnull().sum()

0

- **Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.**

In [22]:
#Primero comprueba cuantos nulos existen en "sex_"
df["type"].isnull().sum()

4

In [23]:
#Primero imputamos con estrategia moda
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
#Donde voy a trabajar
imputer = imputer.fit(df[['type']])
#Aplico la estrategia transformandolo y creando una nueva columna
df['new_type'] = imputer.transform(df[['type']])

In [24]:
#Compruebo si el valor de nulos es ahora 0
df["new_type"].isnull().sum()

0

----------


    Para type y para sex_, lo hemos hecho de forma individual, pero tambien pordemos cambiar mas de una columna a la vez

In [25]:

#Primero imputamos con estrategia moda
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
#directamente aplicamos la estrategia transformandolo y creando una nueva columna
df[['new_type','new_sex']] = imputer.fit_transform(df[['type','sex_']])

----------

- **Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.**

In [26]:
#Comprobamos el tipo de nuestros datos en DF
df.dtypes

year             int64
type            object
country         object
sex_            object
age            float64
species_        object
fatal_(y/n)     object
date            object
new_age        float64
new_sex         object
new_type        object
dtype: object

    Vamos a crear una copia del DF, para mostrar los resultados de este apartado sin que nuestro DF original sufra los cambios, para poder realizar el próximo ejercicio. 😄

In [27]:
df_copia=df.copy()

In [28]:
#Seleccionamos las numéricas
numericas = df_copia.select_dtypes(include = np.number)

In [29]:
#Vemos cuales las variables numéricas de nuestro DF
numericas.head(1)

,year,age,new_age
0,2018,57.0,57.0


In [30]:
#Comprobamos los nulos de nuestras numéricas
numericas.isnull().sum()

year          0
age        2807
new_age       0
dtype: int64

In [31]:
# inciamos el KNNImputer 
imputerKNN = KNNImputer(n_neighbors=5)
# lo aplicamos a nuestras variables numéricas
imputerKNN.fit(numericas)
#Transformamos y lo almacenamos en una variable
numericas_knn= imputerKNN.transform(numericas)
#Lo convertimos en DF
df_knn_imputer = pd.DataFrame(numericas_knn, columns = numericas.columns)

In [32]:
df_knn_imputer.head()

,year,age,new_age
0,2018.0,57.0,57.000000
1,2018.0,11.0,11.000000
2,2018.0,48.0,48.000000
3,2018.0,27.6,27.329155
4,2018.0,27.6,27.329155


In [33]:
# Sacamos el nombre de las columnas del dataframe sin nulos
columnas_knn = df_knn_imputer.columns
# Ahora borramos estas columnas de nuestro DF original para suplirlas por "columnas_knn"
df_copia.drop(columnas_knn, axis = 1, inplace = True)
df_copia[columnas_knn] = numericas_knn

In [34]:
df_copia.head()

,type,country,sex_,species_,fatal_(y/n),date,new_sex,new_type,year,age,new_age
0,Boating,usa,F,white_shark,N,25-Jun-2018,F,Boating,2018.0,57.0,57.000000
1,Unprovoked,usa,F,unspecified,N,18-Jun-2018,F,Unprovoked,2018.0,11.0,11.000000
2,Invalid,usa,M,unspecified,N,09-Jun-2018,M,Invalid,2018.0,48.0,48.000000
3,Unprovoked,australia,M,unspecified,N,08-Jun-2018,M,Unprovoked,2018.0,27.6,27.329155
4,Provoked,mexico,M,tiger_shark,N,04-Jun-2018,M,Provoked,2018.0,27.6,27.329155


In [35]:
df_copia.isnull().sum()

type             4
country         49
sex_           562
species_         0
fatal_(y/n)    532
date             0
new_sex          0
new_type         0
year             0
age              0
new_age          0
dtype: int64

- **Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.**

In [36]:
df = pd.read_csv("./limpio con nan.csv", index_col=0).reset_index()
df

,index,year,type,country,sex_,age,species_,fatal_(y/n),date
0,0,2018,Boating,usa,F,57.0,white_shark,N,25-Jun-2018
1,1,2018,Unprovoked,usa,F,11.0,unspecified,N,18-Jun-2018
2,2,2018,Invalid,usa,M,48.0,unspecified,N,09-Jun-2018
3,3,2018,Unprovoked,australia,M,NaN,unspecified,N,08-Jun-2018
4,4,2018,Provoked,mexico,M,NaN,tiger_shark,N,04-Jun-2018
...,...,...,...,...,...,...,...,...,...
6232,6278,0,Unprovoked,australia,M,NaN,unspecified,Y,Before 1906
6233,6279,0,Unprovoked,australia,M,NaN,unspecified,Y,Before 1903
6234,6281,0,Unprovoked,usa,M,NaN,unspecified,Y,1900-1905
6235,6282,0,Unprovoked,panama,M,NaN,unspecified,Y,1883-1889


In [37]:
df_copia2=df.copy()

In [38]:
#Nuestras columnas numéricas
numericas = df_copia2.select_dtypes(include = np.number)
numericas.head(1)

,index,year,age
0,0,2018,57.0


In [39]:
#Cuantos nulos encontramos
numericas.isnull().sum()

index       0
year        0
age      2807
dtype: int64

In [40]:
#Calculamos su media
mean= numericas.mean()
mean

index    3133.003848
year     1928.874940
age        27.329155
dtype: float64

In [41]:
# Vamos a utilizar el método Interative y señalamos las caracteristicas que queremos.
imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending')
# lo aplicamos y transformamos sobre nuestras variables numéricas, para que se reemplacen los valores nulos.
imputer.fit(numericas)
# transformamos nuestros datos, para que se reemplacen los valores nulos
imputer.transform(numericas)
# convertimos el array que nos devuelve en un dataframe
numericas_trans = pd.DataFrame(imputer.transform(numericas), columns = numericas.columns)

In [42]:
numericas_trans.head()

,index,year,age
0,0.0,2018.0,57.000000
1,1.0,2018.0,11.000000
2,2.0,2018.0,48.000000
3,3.0,2018.0,32.052499
4,4.0,2018.0,32.050581


In [43]:
# lo primero que hacemos es sacar el nombre de las columnas del dataframe sin nulos
columnas = numericas_trans.columns

In [44]:
# utilizando "columnas" eliminamos esas columnas de nuestro dataframe
df_copia2.drop(columnas, axis = 1, inplace = True)

In [45]:
# creamos nuevas columnas en nuestro dataframe original basándonos en el dataframe de las numericas_trans
df_copia2[columnas] = numericas_trans[columnas]

In [46]:
df_copia2.isnull().sum()

type             4
country         49
sex_           562
species_         0
fatal_(y/n)    532
date             0
index            0
year             0
age              0
dtype: int64

In [47]:
numericas_trans.isnull().sum()

index    0
year     0
age      0
dtype: int64

- **¿Podríais explicar que diferencia hay entre estos dos últimos métodos?**



    IterativeImputer: Solo para valores numéricos. Itera por todo el dataframe para obtener el valor aproximado, tantas veces como n-iteraciones se hayan seleccionado.
    
    KNNImputer: Solo para valores numéricos o fechas. Observa para los n-vecinos más proximos que se hayan seleccionado


# EJERCICIO 2

- **Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair**

In [48]:
df_copia.to_csv("limpieza_5.csv") #hemos guardado es con knn donde las numericas han sido sustituidas por nuevo DF limpio sin creación de una columna nueva